# SageMaker를 사용하여 ML 모델 평가하기

![training notebook](img/sagemaker-mlops-model-evaluation-diagram.jpg)

In [ ]:
!pip install scikit-learn s3fs==0.4.2 sagemaker xgboost mlflow==2.13.2 sagemaker-mlflow==0.1.0

관련 라이브러리 가져오기

In [ ]:
from model_evaluation import evaluation
import boto3
import sagemaker

# 변수 초기화

In [ ]:
region = "us-east-1"
boto_session = boto3.Session(region_name=region)
sess = sagemaker.Session(boto_session=boto_session)
bucket_name = sess.default_bucket()
bucket_prefix = "player-churn/xgboost"
notebook_param_s3_bucket_prefix=f"s3://{bucket_name}/{bucket_prefix}/params"
experiment_name = "player-churn-model-experiment"
mlflow_tracking_server_arn = "" # Provide a valid mlflow tracking server ARN. You can find the value in the output from 00-start-here.ipynb
run_id = None

In [ ]:
assert len(mlflow_tracking_server_arn) > 0

# 예측 결과 해석하기
많은 이진 분류 알고리즘의 실제 출력은 예측 점수입니다. 이 점수는 주어진 관측치가 양성 클래스(실제 타겟 값이 1)에 속한다는 시스템의 확신도를 나타냅니다. 이진 분류 모델은 0에서 1 사이의 점수를 출력합니다. 이 점수를 사용하여 관측치를 1 또는 0으로 분류할지 결정하려면, 분류 임계값 또는 컷오프를 선택하고 점수와 비교하여 해석합니다. 임계값보다 높은 점수를 가진 관측치는 타겟=1로 예측되고, 임계값보다 낮은 점수는 타겟=0으로 예측됩니다.

기본 점수 컷오프는 0.5입니다. 비즈니스 요구사항에 맞게 이 컷오프를 업데이트할 수 있습니다. 콘솔의 시각화를 사용하여 컷오프 선택이 애플리케이션에 어떤 영향을 미치는지 이해할 수 있습니다.

## ML 모델 정확도 측정하기
이진 분류 모델의 업계 표준 정확도 지표는 ROC(Receiver Operating Characteristic) 곡선 아래 면적(AUC)이라고 합니다. AUC는 모델이 음성 예제와 비교하여 양성 예제에 대해 더 높은 점수를 예측하는 능력을 측정합니다. 점수 컷오프와 독립적이기 때문에, 임계값을 선택하지 않고도 AUC 지표를 통해 모델의 예측 정확도를 파악할 수 있습니다.

AUC 지표는 0에서 1 사이의 소수값을 반환합니다. 1에 가까운 AUC 값은 ML 모델이 매우 정확하다는 것을 나타냅니다. 0.5에 가까운 값은 ML 모델이 무작위 추측보다 나을 것이 없다는 것을 나타냅니다. 0에 가까운 값은 보기 드물며, 일반적으로 데이터에 문제가 있음을 나타냅니다. 본질적으로 0에 가까운 AUC는 ML 모델이 올바른 패턴을 학습했지만, 현실과 반대로 예측을 하고 있다는 것을 의미합니다('0'은 '1'로, '1'은 '0'으로 예측됨). AUC에 대한 자세한 내용은 위키피디아의 [Receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) 페이지를 참조하세요.

이진 모델의 기준 AUC 지표는 0.5입니다. 이는 무작위로 1 또는 0 답변을 예측하는 가상의 ML 모델의 값입니다. 이진 ML 모델은 가치 있게 되기 위해 이 값보다 더 나은 성능을 보여야 합니다.

In [ ]:
auc_score = evaluation.evaluate(region,
             bucket_prefix,
             mlflow_tracking_server_arn,
             experiment_name,
             run_id)

In [ ]:
print(f"Model Evaluation: AUC Score: {auc_score}")